In [91]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.common.exceptions import TimeoutException

from bs4 import BeautifulSoup, Comment
import re
import pandas as pd
import os

from urllib.parse import urlparse, parse_qs
import os.path

import time

import nltk
from nltk import sentiment
from nltk.corpus import opinion_lexicon
from nltk.tokenize import treebank

In [101]:
# nltk.download('maxent_ne_chunker')
# nltk.download('averaged_perceptron_tagger')

In [92]:
a = ne_chunk(pos_tag(word_tokenize("Suck my cock")))

In [102]:
def load_facebook():
    url = "https://facebook.com"
    driver = webdriver.Firefox()
    driver.get(url)
    driver.find_element_by_id('email').send_keys('email')
    driver.find_element_by_id('pass').send_keys('password')
    driver.find_element_by_id('pass').send_keys(Keys.ENTER)
    
    try:
        elem = WebDriverWait(driver, 2).until(EC.title_contains("Facebook"))
    except TimeoutException:
        print("Too much time")
    
    return driver

def get_nltk_sentiment(sentence, method):
        
    if (method == 'vader'):
        sa = sentiment.vader.SentimentIntensityAnalyzer()
        output = sa.polarity_scores(str(sentence))

        return output['compound']
    elif (method == 'liu'):
        #if name, return positive
        
        if "PERSON" in str(ne_chunk(pos_tag(word_tokenize("hey pretty cool")))):
            return 'Positive'
        
        tokenizer = treebank.TreebankWordTokenizer()
        pos_words = 0
        neg_words = 0
        tokenized_sent = [word.lower() for word in tokenizer.tokenize(sentence)]
        
        for word in tokenized_sent:
            if word in opinion_lexicon.positive():
                pos_words += 1
            elif word in opinion_lexicon.negative():
                neg_words += 1
        if pos_words > neg_words:
            return 'Positive'
        elif pos_words < neg_words:
            return 'Negative'
        elif pos_words == neg_words:
            return 'Neutral'

def get_comments(url, driver):
    
    pagePath = urlparse(url).path
    pName = pagePath.split("/")[1]
    
    driver.get(url)
    
    try:
        elem = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.CLASS_NAME, "fbPhotoSnowliftContainer")))
    except TimeoutException:
        print("Too much time")
    
    while True:
        time.sleep(2)
        
        try:
            driver.find_elements_by_xpath("(//a[contains(text(),'View more comments')])")[3].click()
        except:
            break
            
    html = driver.page_source

    soup = BeautifulSoup(html, "lxml")

    comments = soup.find_all("div", class_="UFICommentContent")

    df = pd.DataFrame(columns=['Comment', 'Profile URL'])

    for comment in comments:
        body = comment.find("span", class_="UFICommentBody").getText()
        currURL = comment.find('a', href=True)['href']
        
        if (currURL.find(pName) == -1):   
            df = df.append({'Comment': body, 'Profile URL': currURL}, ignore_index=True)
            

    return df

def get_positive(url, driver):
    df = get_comments(url, driver)
    df['Sentiment'] = df['Comment'].apply(get_nltk_sentiment, method="liu")
    newDf = df[df['Sentiment'] == 'Positive'].drop_duplicates('Profile URL', keep='first')
    
    newDf = newDf.reset_index(drop=True)
    return newDf

def get_id(text):
    parsed = urlparse(text)
    parsed_dict = parse_qs(parsed.query)
    userId = parsed_dict['id']

    return userId[0]

def get_userid(url, driver):
    
    if ("profile.php?id=" in url):
        text = url
        ret_id = get_id(text)
    else:
        driver.get(url)

        try:
            elem = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.CLASS_NAME, 'hidden_elem')))
        except TimeoutException:
            print("Too much time")

        html = driver.page_source
        soup = BeautifulSoup(html, "lxml")

        userId = ''

        for aVals in soup.find_all('a'):
            try:
                text = aVals['data-hovercard']
                ret_id = get_id(text)
                
                if ("/ajax/hovercard/user.php?id={}".format(ret_id) in text):
                    print(text)
                    break
            except:
                pass
            
    return ret_id

In [ ]:
driver = load_facebook()

In [ ]:
df = get_positive('https://www.facebook.com/promisedlanddairy/photos/a.328814060485906.84135.179461015421212/1896136953753601/?type=3&theater', driver)

In [132]:
df['ProfileID'] = ""

for idx, row in df.iterrows():
    df.loc[idx]['ProfileID'] = get_userid(row['Profile URL'], driver)

/ajax/hovercard/user.php?id=100000670838598
/ajax/hovercard/user.php?id=539392911
/ajax/hovercard/user.php?id=1684836462
/ajax/hovercard/user.php?id=100014371653039
/ajax/hovercard/user.php?id=1235595588
/ajax/hovercard/user.php?id=1075832673
/ajax/hovercard/user.php?id=100009881200476
/ajax/hovercard/user.php?id=100006415140770
/ajax/hovercard/user.php?id=100013041578845
/ajax/hovercard/user.php?id=100006221528948
/ajax/hovercard/user.php?id=100000569083610
/ajax/hovercard/user.php?id=100009668298881
/ajax/hovercard/user.php?id=100005646499164
/ajax/hovercard/user.php?id=100022698270187
/ajax/hovercard/user.php?id=1492300824
/ajax/hovercard/user.php?id=1381142216
/ajax/hovercard/user.php?id=100014341125065
/ajax/hovercard/user.php?id=100022435635286
/ajax/hovercard/user.php?id=100014470786149
/ajax/hovercard/user.php?id=1425584277
/ajax/hovercard/user.php?id=100000245268913
/ajax/hovercard/user.php?id=100024712938956
/ajax/hovercard/user.php?id=100007430049121
/ajax/hovercard/user.php

KeyboardInterrupt: 

In [134]:
def scrape_intersection(soup):
    res = soup.find("div", {"id": "BrowseResultsContainer"})

    links = res.find_all('a')

    linkList = []

    for link in links:
        if ('facebook.com' in link['href']):
            if (link['href'] not in linkList):
                linkList.append(link['href'])
                
    return linkList

In [138]:
def perform_intersection(df, driver):
    intersectURL = 'https://www.facebook.com/search/{}/pages-liked/{}/pages-liked/intersect'
    
    df = pd.DataFrame(columns=['User 1', 'User 2', 'Common Interests'])
    
    for idx, row in df.iterrows():
        emptyNo = 0
    
        for idx1, otherrow in tDf.iterrows():
            if (row['ProfileID'] != otherrow['ProfileID']):

                if (emptyNo >= 6):
                    print('Breaking because last 6 requests were empty')
                    break

                newURL = intersectURL.format(row['ProfileID'], otherrow['ProfileID'])
                driver.get(newURL)

                try:
                    elem = WebDriverWait(driver, 2).until(EC.title_contains("Pages liked by"))
                except TimeoutException:
                    print("Too much time")

                time.sleep(3)

                html = driver.page_source
                soup = BeautifulSoup(html, "lxml")

                if "We couldn\'t find anything for" in str(soup):
                    print("No intersect found")
                    emptyNo = emptyNo + 1
                else:
                    lst = scrape_intersection(soup)
                    df = df.append({'User 1': row['ProfileID'], 'User 2': otherrow['ProfileID'], 'Common Interests': lst}, ignore_index=True)
                    
                    print(df)
                    
                    emptyNo = -1000
                    
    return df

In [140]:
#probably another function to isolate url

In [5]:
commonDf = perform_intersection(df[:20], driver)

In [ ]:
#https://www.facebook.com/search/100000670838598/pages-liked/100017761387060/pages-liked/intersect